In [51]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from ipywidgets import interact, FloatSlider
from scipy.stats import norm
import time

# Variance Gamma Model with Brownian Motion

$$ X(t; \eta, \beta, \theta) = \theta \Gamma(t; 1, \beta) + \eta W(\Gamma(t; 1, \beta)) + \sigma Bt $$
- $\Gamma(t; 1, \beta)$: Gamma process with mean $\frac{t}{k}$ and variance $\beta$
- $W(\Gamma(t; 1, \beta))$: Brownian motion with gamma clock 
- $\eta$ : volatility of Brownian motion with gamma clock 
- Bt : Brownian motion 
- $\theta$ : drift parameter

$$\eta \ge 0$$
$$ \theta + \frac{\eta^2}{2} < \frac{1}{\beta}$$



In [52]:
@dataclass
class VGBparams:
    beta: float
    eta: float
    sigma: float
    theta: float

In [53]:
def VGB_cf_log_ST(u, S0, T, r, q, p:VGBparams):
    omega = (1.0/p.beta) * np.log(1 - p.beta * p. theta - 0.5 * p.beta * p.eta ** 2)
    first_term = np.exp(1j * u * np.log(S0))
    drift_term = np.exp(1j * u * (r - q + omega - 0.5 * p.sigma ** 2) * T)
    diff_term = np.exp(-0.5 * p.sigma ** 2 * u ** 2 * T)
    extra_term = (1.0 - 1j * p.beta * p.theta * u + 0.5 * p.beta * p.eta ** 2 * u ** 2) ** (-T/p.beta)
    return first_term * drift_term * diff_term * extra_term

def VGB_cf_ST(u, S0, T, r, q, p:VGBparams):
    omega = (1.0/p.beta) * np.log(1 - p.beta * p. theta - 0.5 * p.beta * p.eta ** 2)
    drift_term = np.exp(1j * u * (r - q + omega - 0.5 * p.sigma ** 2) * T)
    diff_term = np.exp(-0.5 * p.sigma ** 2 * u ** 2 * T)
    extra_term = (1.0 - 1j * p.beta * p.theta * u + 0.5 * p.beta * p.eta ** 2 * u ** 2) ** (-T/p.beta)
    return drift_term * diff_term * extra_term


def simpson_weights(N: int):
    if N % 2 != 0:
        raise ValueError("N must be even")
    w = np.ones(N)
    w[1:N-1:2] = 4
    w[2:N-2:2] = 2
    return w

# alpha dampening factor
# eta is step in frequency domain

def vgb_fft_calls(S0:float, T:float, r:float, q:float, p:VGBparams, N:int = 2**14, eta:float = 0.1, alpha:float = 1.5):
    n = np.arange(N)
    v = eta * n   
    i = 1j
    phi_shift = VGB_cf_log_ST(v - (alpha + 1)*i, S0, T , r, q, p)        
    denom = (alpha ** 2 + alpha  - v ** 2 + i*(2*alpha + 1)* v)  
    psi = np.exp(-r * T) * phi_shift/denom

    w = simpson_weights(N) * (eta / 3.0)

    # FFT coupling
    lam  = 2.0 * np.pi / (N * eta)   # Δk (log-strike step)
    b = 0.5 * N * lam     # half-width in k
    x   = psi * np.exp(1j * b * v) * w

    F  = np.fft.fft(x)
    F  = np.real(F)

    j = np.arange(N)
    k = -b + j * lam                 # k = ln K
    K = np.exp(k)

    calls = np.exp(-alpha * k) / np.pi * F
    order = np.argsort(K)
    return K[order], np.maximum(calls[order], 0.0)


def vgb_fft_call_price(S0: float, K: float, T: float, r: float, q: float, p: VGBparams,
    N: int = 2**14, eta: float = 0.1, alpha: float = 1.5):
    K_grid, C_grid = vgb_fft_calls(S0, T, r, q, p, N=N, eta=eta, alpha=alpha)
    if K <= K_grid[0]:
        return C_grid[0]
    if K >= K_grid[-1]:
        return C_grid[-1]
    idx = np.searchsorted(K_grid, K)
    x0, x1 = K_grid[idx-1], K_grid[idx]
    y0, y1 = C_grid[idx-1], C_grid[idx]
    return y0 + (y1 - y0) * (K - x0) / (x1 - x0)

def vgb_fft_put_price(
    S0: float, K: float, T: float, r: float, q: float, p: VGBparams,
    N: int = 2**14, eta: float = 0.1, alpha: float = 1.5
):
    """Put via put–call parity."""
    C = vgb_fft_call_price(S0, K, T, r, q, p, N=N, eta=eta, alpha=alpha)
    return C - S0*np.exp(-q*T) + K*np.exp(-r*T)

In [54]:

def cos_price_call(S0, K, T, r, q, p:VGBparams, N=2**15, L = 15):
    a = 0 - L * np.sqrt(T)
    b = 0 + L * np.sqrt(T)
    x0 = np.log(S0/K)
    k = np.arange(N)                      # 0,1,...,N-1
    u = k * np.pi / (b - a)               # frequencies

   


    # Compute Vk coefficients analytically

    c = 0
    d = b                                

    

    # psi_k(c,d)
    psi = np.zeros(N)
    psi[0] = d - c
    psi[1:] = (np.sin(k[1:] * np.pi * (d - a)/(b - a)) - np.sin(k[1:] * np.pi * (c - a)/(b - a))) * (b - a) / (k[1:] * np.pi)

    # chi_k(c,d)
    term1 = 1.0 / (1.0 + (k * np.pi / (b - a)) ** 2)
    term2 = np.cos(k * np.pi  * ((d - a)/ (b - a))) * np.exp(d)
    term3 = np.cos(k * np.pi  * ((c - a)/ (b - a))) * np.exp(c)
    term4 = (k * np.pi / (b - a)) * np.sin(k * np.pi * ((d - a)/(b - a))) * np.exp(d)
    term5 = (k * np.pi / (b - a)) * np.sin(k * np.pi * ((c - a)/(b - a))) * np.exp(c)
    chi = term1 * (term2  - term3 + term4 - term5)

    Vk = 2.0 / (b - a) * (chi - psi)
    Vk[0] *= 0.5  # first term has weight 1/2 in COS expansion

    # Characteristic function part
    temp = (VGB_cf_ST(u, S0, T, r , q, p) * Vk)
    mat = np.exp(1j * (x0 - a) * u)
    value = np.exp(-r * T) * K * np.real(mat @ temp)
    # COS formula
    
    return float(value)



hp = VGBparams(0.14, 0.53, 0.15, 0.11)
start_1 = time.perf_counter()
call_price_1 = vgb_fft_call_price(100.0, 100.0, 0.3, 0.02, 0.0, hp)
t1 = time.perf_counter() - start_1
start_2 = time.perf_counter()
call_price_2 = cos_price_call(100.0, 100.0, 0.3, 0.02, 0, hp)
t2 = time.perf_counter() - start_2
print(call_price_1)
print(f"FFT time: {t1:.6f} s")

print(call_price_2)
print(f"COS time: {t2:.6f} s")

11.944609854859188
FFT time: 0.004692 s
11.94446482371592
COS time: 0.007177 s


In [55]:
def European_Call(S0, K, T, r , q, sigma):
    d1 = (np.log(S0/K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return  S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_vega(S0, K, T, r, q, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * norm.pdf(d1) * np.sqrt(T)

def get_implied_vol(market_price, S0, K, T, r, q, max_iter = 200, tolerance = 1e-05):
    sigma = 0.3
    for _ in range(max_iter):
        bs_price = European_Call(S0, K, T, r, q, sigma)
        diff = bs_price - market_price
        if abs(diff) < tolerance:
            return sigma
        vega = bs_vega(S0,K,T,r, q, sigma)
        if vega == 0:
            return None
        sigma -= diff/vega
    return None

In [56]:

def plot_BS_implied_vol(S0, T, r, q, sigma, beta, eta, theta):
    K_min = S0 * 0.90
    K_max = S0 * 1.05
    fig, ax = plt.subplots(1,2, figsize = (12,6))
    
    hp = VGBparams(beta = beta, eta = eta, sigma = sigma, theta = theta)
    moneyness_grid = np.log(np.linspace(K_min , K_max, 300)/(S0 * np.exp(r * T)))
    K_grid = np.linspace(K_min, K_max, 300)
   
    
    vgb_prices = [vgb_fft_call_price(S0, k, T, r , q, hp) for k in K_grid]
    vgb_COS_prices = [cos_price_call(S0, k, T, r, q, hp) for k in K_grid]
    black_scholes_prices = [European_Call(S0, k, T, r, q, sigma) for k in K_grid]
   
    BS_implied_vols = [get_implied_vol(vgb_price, S0, strike, T, r, q) for strike, vgb_price in zip(K_grid,vgb_prices)] 
    BS_implied_vols_2 = [get_implied_vol(vgb_price, S0, strike, T, r, q) for strike, vgb_price in zip(K_grid,vgb_COS_prices)] 
    ax[0].plot(K_grid, BS_implied_vols, color = 'black', label = "FFT")
    ax[0].plot(K_grid, BS_implied_vols_2, color = 'red', label = "COS")
    ax[0].set_xlabel("Strike")
    ax[0].set_ylabel("BS implied volatility (%)")
    ax[0].set_title("BS implied volatility(%) for VGB model")
    ax[0].grid(True)
    ax[0].legend()
    ax[1].plot(K_grid, vgb_prices, color = 'black', label = "VGB model")
    ax[1].plot(K_grid, black_scholes_prices, color = 'blue', label = "Black_Scholes model")
    ax[1].set_xlabel("Strike")
    ax[1].set_ylabel("Call option price")
    ax[1].set_title("Call option price for VGB model")
    ax[1].grid(True)
    ax[1].legend()
    plt.show()


interact(plot_BS_implied_vol,
         s_min = FloatSlider(value=60, min=20,  max=100, step=1,   description="S_min"),
         s_max = FloatSlider(value=160, min=100, max=300, step=1 , description="S_max"),
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=0.002, min=0.01, max=5.0, step=0.001, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         sigma=FloatSlider(value=0.1511, min=0.01, max=10.0, step=0.01, description="sigma"),
         beta=FloatSlider(value=0.14, min=0.001, max=3.0, step=0.005, description="beta"),
         theta=FloatSlider(value=0.11, min=-0.99, max=0.99, step=0.005, description="theta"),
         eta=FloatSlider(value=0.53, min=0.001, max=5.0, step=0.005, description="eta")
        )
       

interactive(children=(FloatSlider(value=100.0, description='S0', max=300.0, min=20.0, step=1.0), FloatSlider(v…

<function __main__.plot_BS_implied_vol(S0, T, r, q, sigma, beta, eta, theta)>

In [57]:
def risk_neutral_density_log_St(x_array, S0, r, q, T, sigma, beta, eta, theta,
                                u_max=200, n_steps=5000):
    """
    Returns f_{X_T}(x) for X_T = log S_T at points x_array.
    """
    hp = VGBparams(beta = beta, eta = eta, sigma = sigma, theta = theta)

    # integration grid in u
    u = np.linspace(0.0, u_max, n_steps)              # shape (N_u,)
    phi = VGB_cf_log_ST(u, S0, T, r, q, hp)               # shape (N_u,)


    # shape (N_u, N_x): outer product in exponent
    exp_term = np.exp(-1j * np.outer(u, x_array))
    integrand = np.real(exp_term * phi[:, None])      # broadcast phi over columns

    # integrate over u (axis=0) to get f_X(x) for each x
    fx = (1/np.pi) * np.trapezoid(integrand, u, axis=0)
    return fx   # shape (N_x,)


def plot_risk_neutral_density_St(s_min, s_max, S0, r, q, T, beta, eta, sigma, theta,
                                 u_max=200, n_steps=5000):
    """
    Plots f_{S_T}(s) and returns the density values.
    """
    
    s_array = np.linspace(s_min, s_max, 200)
    x_array = np.log(s_array)

    # f_X(x)
    fx = risk_neutral_density_log_St(x_array, S0, r, q, T, sigma, beta, eta, theta, u_max=u_max, n_steps=n_steps)

    fS = fx / s_array

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(s_array, fS, color = 'black')
    ax.axvline(x=S0, color='red', linestyle='--', linewidth=2)
    ax.set_xlabel(r"$s$")
    ax.set_ylabel(r"$f_{S_T}(s)$")
    ax.set_title("Risk-neutral density of $S_T$ under VGB model")
    ax.grid(True)
    plt.show()

interact(plot_risk_neutral_density_St,
         s_min = FloatSlider(value=60, min=20,  max=100, step=1,   description="S_min"),
         s_max = FloatSlider(value=160, min=100, max=300, step=1 , description="S_max"),
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=0.002, min=0.01, max=5.0, step=0.001, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         sigma=FloatSlider(value=0.1511, min=0.01, max=10.0, step=0.01, description="sigma"),
         beta=FloatSlider(value=0.14, min=0.001, max=3.0, step=0.005, description="beta"),
         theta=FloatSlider(value=0.11, min=-0.99, max=0.99, step=0.005, description="theta"),
         eta=FloatSlider(value=0.53, min=0.001, max=5.0, step=0.005, description="eta")

        )


interactive(children=(FloatSlider(value=60.0, description='S_min', min=20.0, step=1.0), FloatSlider(value=160.…

<function __main__.plot_risk_neutral_density_St(s_min, s_max, S0, r, q, T, beta, eta, sigma, theta, u_max=200, n_steps=5000)>